In [3]:
api_key="api key"

In [4]:
from langchain_groq import ChatGroq
import os

from langchain_core.tools import tool
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage, ToolMessage

from datetime import date
import random

@tool
def get_compliance_standard(standard: str):
    """Retrieve details about a specific compliance standard (e.g., GDPR, HIPAA, ISO 27001)."""
    compliance_data = {
        "GDPR": "General Data Protection Regulation ensures data privacy in the EU.",
        "HIPAA": "Health Insurance Portability and Accountability Act regulates healthcare data.",
        "ISO 27001": "International standard for information security management systems (ISMS)."
    }
    return compliance_data.get(standard, "Standard not found.")

def assess_risk(risk_type):
    """
    Provides a basic risk assessment for common business risks.
    """
    risk_levels = ["Low", "Medium", "High"]
    risk_assessment = {
        "cybersecurity": f"Cybersecurity risk is assessed as {random.choice(risk_levels)}. Consider regular security audits.",
        "financial": f"Financial risk is assessed as {random.choice(risk_levels)}. Review budgets and cost management.",
        "operational": f"Operational risk is assessed as {random.choice(risk_levels)}. Ensure process optimization."
    }
    return risk_assessment.get(risk_type.lower(), "Risk type not recognized. Try cybersecurity, financial, or operational.")

@tool
def get_compliance_info(standard: str):
    """Fetch information about a specific compliance standard."""
    return get_compliance_standard(standard)

@tool
def get_risk_assessment(risk_type: str):
    """Provide a basic risk assessment for cybersecurity, financial, or operational risks."""
    return assess_risk(risk_type)

def call_functions(llm_with_tools, user_prompt):
    system_prompt = 'You are a Governance, Risk, and Compliance (GRC) consultant assisting organizations. Today is {today}'.format(today=date.today())
    messages = [SystemMessage(system_prompt), HumanMessage(user_prompt)]

    ai_msg = llm_with_tools.invoke(messages)
    messages.append(ai_msg)

    for tool_call in ai_msg.tool_calls:
        selected_tool = {
            "get_compliance_info": get_compliance_info,
            "get_risk_assessment": get_risk_assessment
        }.get(tool_call["name"], None)

        if selected_tool:
            tool_output = selected_tool.invoke(tool_call["args"])
            messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

    return llm_with_tools.invoke(messages).content

def main():
    llm = ChatGroq(groq_api_key="api key", model='llama3-70b-8192')
    tools = [get_compliance_info, get_risk_assessment]
    llm_with_tools = llm.bind_tools(tools)

    print("Welcome to the GRC Consultant Agent!")
    print("Ask about compliance standards (e.g., 'Tell me about GDPR') or risk assessments (e.g., 'Assess financial risk').")

    while True:
        user_question = input("\nEnter your question (or type 'exit' to quit): ")
        if user_question.lower() == 'exit':
            break
        response = call_functions(llm_with_tools, user_question)
        print("\nGRC Agent:", response)

if __name__ == "__main__":
    main()


Welcome to the GRC Consultant Agent!
Ask about compliance standards (e.g., 'Tell me about GDPR') or risk assessments (e.g., 'Assess financial risk').

Enter your question (or type 'exit' to quit): what are iso standards ?


<ipython-input-4-f95fdd430901>:35: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return get_compliance_standard(standard)



GRC Agent: ISO is a broad term that encompasses a wide range of standards. Could you please be more specific about which ISO standard you're interested in? For example, are you looking for information on ISO 27001 (information security), ISO 31000 (risk management), or perhaps ISO 19600 (compliance management)?

Enter your question (or type 'exit' to quit): give some iso standard examples for  risk management

GRC Agent:  They provide a structured approach to identifying, assessing, and mitigating risks, and can help organizations to improve their overall risk management capabilities.

Let me know if you have any further questions!

Enter your question (or type 'exit' to quit): give iso standard examples for risk management

GRC Agent: 

Enter your question (or type 'exit' to quit): exit
